# pycrpto

In [24]:
import json
from Crypto.Cipher import AES

#秘钥,此处需要将字符串转为字节
key = b'0e9ec82fc54142de919f557a28bf637a'

#加密内容需要长达16位字符，所以进行空格拼接
def pad(text):
  while len(text) % 16 != 0:
    text += b' '
  return text

#加密秘钥需要长达16位字符，所以进行空格拼接
def pad_key(key):
  while len(key) % 16 != 0:
    key += b' '
  return key

#进行加密算法，模式ECB模式，把叠加完16位的秘钥传进来
aes = AES.new(pad_key(key), AES.MODE_ECB)
aa = {"action":"0","address":"","area":"","city":"","entContact":"daisy","entContactEmail":"ying.chen@tongfudun1111.com","entContactPhone":"18896551111","entId":810,"entName":"通付盾1111","entNo":"tongfudun1111","industry":16,"organizationId":"7d5d12318cf644bab92b16f0eebaf52c","orgnazationId":"7d5d12318cf644bab92b16f0eebaf52c","platform":"B","province":""}
#加密内容,此处需要将字符串转为字节
text=b'thisisjiamineirong'
# text = str(json.dumps(aa))
#进行内容拼接16位字符后传入加密类中，结果为字节类型
encrypted_text = aes.encrypt(pad(text))

print(encrypted_text)

#此处是为了验证是否能将字节转为字符串后，进行解密成功
#实际上a 就是 encrypted_text ，也就是加密后的内容
a = b'\x91\xbc\x7f*\xae\x02\xf8\x0eB\xc8\x98\x03z\xbb5@<(\xa1\x90\xad\x14veyn\x18\x18\xb5\xf7,\x9b'
# a = b'FB7DB864394160B719DA655CEB7400A7A5995A8DE187A2FA87BEB06CD3239CEEFBB19009818AF99DA71FB6A750537624665A8A0CA1AFE144A85F4C4475C413E81952D502722B7B6551F92A92C7C19644BC376D9D355C5F072C45F52F80FED8B5C25C828859BBB0F2F400249C851E96A2CD639F2A6E47305BB53B3EF1E77C80378D74E9E7FF663EFA610CCBB432D2163BC304EED03B56970F464B0FC8000EBA1DAF223D72D2A66A9994A370DBBC168A208666B287EAF06C4410D38C14FF78A1E802BFAB1AF7C06F5C849841827B78AF661C57154C758904B2305CA225F5DD6D952F90AB4059A4884C59BB5A5BFFD5722E52E4C76DA65BF1A7A95ECB99E1A03294223093E826F8B23C91342A1F1948CABE139503F4767B96ABAC2C89ED608F34749C107071B3B8588E38B43CA0036E879E382D490AB71F460575035FF965E7E69351A51F3A343A3C986AB6622112EE4FDE38CD0269AF5CEB7264222B365D81631D1FB96B52147F892B00CB3347AEAE2961'
#用aes对象进行解密，将字节类型转为str类型，错误编码忽略不计
de = str(aes.decrypt(a),encoding='utf-8',errors="ignore")

#获取str从0开始到文本内容的字符串长度。
print(de[:len(text)])


b'\x91\xbc\x7f*\xae\x02\xf8\x0eB\xc8\x98\x03z\xbb5@<(\xa1\x90\xad\x14veyn\x18\x18\xb5\xf7,\x9b'
thisisjiamineirong


In [48]:
import json
from Crypto.Cipher import AES

#秘钥,此处需要将字符串转为字节
key = b'0e9ec82fc54142de919f557a28bf637a'

#加密内容需要长达16位字符，所以进行空格拼接
def pad(text):
  while len(text) % 16 != 0:
    text += b' '
  return text

#加密秘钥需要长达16位字符，所以进行空格拼接
def pad_key(key):
  while len(key) % 16 != 0:
    key += b' '
  return key

#进行加密算法，模式ECB模式，把叠加完16位的秘钥传进来
aes = AES.new(pad_key(key), AES.MODE_ECB)


#此处是为了验证是否能将字节转为字符串后，进行解密成功
#实际上a 就是 encrypted_text ，也就是加密后的内容
tt = 'FB7DB864394160B719DA655CEB7400A7A5995A8DE187A2FA87BEB06CD3239CEEFBB19009818AF99DA71FB6A750537624665A8A0CA1AFE144A85F4C4475C413E81952D502722B7B6551F92A92C7C19644BC376D9D355C5F072C45F52F80FED8B5C25C828859BBB0F2F400249C851E96A2CD639F2A6E47305BB53B3EF1E77C80378D74E9E7FF663EFA610CCBB432D2163BC304EED03B56970F464B0FC8000EBA1DAF223D72D2A66A9994A370DBBC168A208666B287EAF06C4410D38C14FF78A1E802BFAB1AF7C06F5C849841827B78AF661C57154C758904B2305CA225F5DD6D952F90AB4059A4884C59BB5A5BFFD5722E52E4C76DA65BF1A7A95ECB99E1A03294223093E826F8B23C91342A1F1948CABE139503F4767B96ABAC2C89ED608F34749C107071B3B8588E38B43CA0036E879E382D490AB71F460575035FF965E7E69351A51F3A343A3C986AB6622112EE4FDE38CD0269AF5CEB7264222B365D81631D1FB96B52147F892B00CB3347AEAE2961'
# 字符串转bytes
ss = bytes.fromhex(tt)

ss = pad(ss)

print(ss)


print("=============")
print(aes.decrypt(ss))
#用aes对象进行解密，将字节类型转为str类型，错误编码忽略不计
de = str(aes.decrypt(ss), encoding='utf-8', errors='ignore')

#获取str从0开始到文本内容的字符串长度。
print(de[:len(text)])


b'\xfb}\xb8d9A`\xb7\x19\xdae\\\xebt\x00\xa7\xa5\x99Z\x8d\xe1\x87\xa2\xfa\x87\xbe\xb0l\xd3#\x9c\xee\xfb\xb1\x90\t\x81\x8a\xf9\x9d\xa7\x1f\xb6\xa7PSv$fZ\x8a\x0c\xa1\xaf\xe1D\xa8_LDu\xc4\x13\xe8\x19R\xd5\x02r+{eQ\xf9*\x92\xc7\xc1\x96D\xbc7m\x9d5\\_\x07,E\xf5/\x80\xfe\xd8\xb5\xc2\\\x82\x88Y\xbb\xb0\xf2\xf4\x00$\x9c\x85\x1e\x96\xa2\xcdc\x9f*nG0[\xb5;>\xf1\xe7|\x807\x8dt\xe9\xe7\xfff>\xfaa\x0c\xcb\xb42\xd2\x16;\xc3\x04\xee\xd0;V\x97\x0fFK\x0f\xc8\x00\x0e\xba\x1d\xaf"=r\xd2\xa6j\x99\x94\xa3p\xdb\xbc\x16\x8a \x86f\xb2\x87\xea\xf0lD\x10\xd3\x8c\x14\xffx\xa1\xe8\x02\xbf\xab\x1a\xf7\xc0o\\\x84\x98A\x82{x\xaff\x1cW\x15Lu\x89\x04\xb20\\\xa2%\xf5\xddm\x95/\x90\xab@Y\xa4\x88LY\xbbZ[\xff\xd5r.R\xe4\xc7m\xa6[\xf1\xa7\xa9^\xcb\x99\xe1\xa02\x94"0\x93\xe8&\xf8\xb2<\x914*\x1f\x19H\xca\xbe\x13\x95\x03\xf4v{\x96\xab\xac,\x89\xed`\x8f4t\x9c\x10pq\xb3\xb8X\x8e8\xb4<\xa0\x03n\x87\x9e8-I\n\xb7\x1fF\x05u\x03_\xf9e\xe7\xe6\x93Q\xa5\x1f:4:<\x98j\xb6b!\x12\xeeO\xde8\xcd\x02i\xaf\\\xebrd"+6]\x81c\x1d\x1f\xb9kR\x14\x7